# 3D Shape Estimator - Feasible Adam
A test run.

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')
project_path = '/content/gdrive/MyDrive/master/3d_shapes_estimator'
sys.path.append(project_path)

Mounted at /content/gdrive


In [2]:
!nvidia-smi
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Sun Aug 28 04:51:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%cd {project_path}

/content/gdrive/MyDrive/master/3d_shapes_estimator


In [4]:
# import DL modules
import tensorflow as tf
import keras as keras
from keras.models import Sequential, Model
from keras import layers
from keras import optimizers
from keras.models import load_model
import keras.backend as K

# import service modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
sys.path.append(os.getcwd()+'/utils')
import binvox_rw
import json

In [5]:
#@title Setup
RUN_MODE = 'test'
OUTPUT_PATH = project_path+"/intermediate/shape_estimator_normal_feasible_test_adam/"
RANDOM_SEED = 10
IS_SAMPLE = False #@param
SAMPLE_RATE = 4
N_EPOCH = 25 #@param {type: "integer"}
BATCH_SIZE = 12 #@param {type: "integer"}
VERBOSE = 1 #@param {type: "boolean"}
GEN_LR = 0.00005
DISC_LR = 0.00005
OPTIMIZER = 'Adam'
GP_WEIGHT = 10.0
PREDICT_SKETCH = False
SHUFFLE = False
np.random.seed(RANDOM_SEED)

In [6]:
tf.__version__

'2.8.0'

# Load Data

In [7]:
# reference: https://keras.io/examples/vision/depth_estimation/
class DataLoader(tf.keras.utils.Sequence):
  def __init__(self, data, batch_size=15, shuffle=False):
    """
    Initialization
    """
    self.shuffle = shuffle
    self.data = data
    self.x1_file = self.data[0]
    self.x2_file = self.data[1]
    self.y_file = self.data[2]
    self.x0_file = self.data[3]
    self.indices = list(range(self.x1_file.len()))
    self.len_indices = len(self.indices)
    self.batch_size = batch_size
    self.on_epoch_end()

  def __len__(self):
    if IS_SAMPLE == True:
      return int(np.ceil(self.len_indices / self.batch_size / SAMPLE_RATE))
    else:
      return int(np.ceil(self.len_indices / self.batch_size))

  def __getitem__(self, index):
    # modify batch size of last batch
    batch_size = self.batch_size
    if (index + 1) * self.batch_size > len(self.indices):
      batch_size = len(self.indices) - index * self.batch_size
    # Generate one batch of data
    # Generate indices of the batch
    index = self.indices[index * batch_size : (index + 1) * batch_size]
    # Find list of IDs
    x1, x2, y, x0 = self.load_batch(index)
    return x1, x2, y, x0


  def on_epoch_end(self):
    """
    Updates indexes after each epoch
    """
    self.index = np.arange(len(self.indices))
    if self.shuffle == True:
        np.random.shuffle(self.index)

  def load_batch(self, batch):
    """
    Load one batch of data.
    """
    x1 = self.x1_file[batch[0]:batch[-1]+1]
    x2 = self.x2_file[batch[0]:batch[-1]+1]
    y = (self.y_file[batch[0]:batch[-1]+1]).astype(np.float16)
    x0 = self.x0_file[batch[0]:batch[-1]+1]
    return x1, x2, y, x0

In [8]:
import h5py

In [9]:
hdf5_path = '/content/gdrive/MyDrive/master/Data/half_data_shuffled.hdf5'
if PREDICT_SKETCH == True:
  train_dataset_file = (h5py.File(hdf5_path)['train_raw_image'], h5py.File(hdf5_path)['train_shape'])
  test_dataset_file = (h5py.File(hdf5_path)['test_raw_image'], h5py.File(hdf5_path)['test_shape'])
  validation_dataset_file = (h5py.File(hdf5_path)['validation_raw_image'], h5py.File(hdf5_path)['validation_shape'])
else:
  train_dataset_file = (h5py.File(hdf5_path)['train_normal'], h5py.File(hdf5_path)['train_depth'], h5py.File(hdf5_path)['train_shape'], h5py.File(hdf5_path)['train_raw_image'])
  test_dataset_file = (h5py.File(hdf5_path)['test_normal'], h5py.File(hdf5_path)['test_depth'], h5py.File(hdf5_path)['test_shape'], h5py.File(hdf5_path)['test_raw_image'])
  validation_dataset_file = (h5py.File(hdf5_path)['validation_normal'], h5py.File(hdf5_path)['validation_depth'], h5py.File(hdf5_path)['validation_shape'], h5py.File(hdf5_path)['validation_raw_image'])

In [10]:
train_loader = DataLoader(data=train_dataset_file, batch_size=BATCH_SIZE, shuffle=False)
validation_loader = DataLoader(data=validation_dataset_file, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(data=test_dataset_file, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
print(train_loader.__len__())
print(validation_loader.__len__())
print(test_loader.__len__())

1434
180
180


In [12]:
sample = next(iter(train_loader))

In [13]:
sample[1].shape

(12, 256, 256, 3)

In [14]:
result = np.concatenate((sample[0], sample[1]), axis=3)

In [15]:
result.shape

(12, 256, 256, 6)

# Define Model

## Generator

In [16]:
# reference: https://towardsdev.com/implement-resnet-with-tensorflow2-4ee81e33a1ac
class ResBlock(layers.Layer):
  def __init__(self, filters, downsample):
    super().__init__()
    if downsample:
      self.conv1 = layers.Conv2D(filters, kernel_size=3, strides=2, padding='same', kernel_initializer = 'he_normal')
      self.shortcut = keras.Sequential([
        layers.Conv2D(filters, kernel_size=1, strides=2, kernel_initializer='he_normal'),
        layers.BatchNormalization()
      ])
    else:
      self.conv1 = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer = 'he_normal')
      self.shortcut = keras.Sequential()

    self.bn1 = layers.BatchNormalization()
    self.rl1 = layers.LeakyReLU(alpha=0.2)

    self.conv2 = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer = 'he_normal')
    self.bn2 = layers.BatchNormalization()
    self.rl2 = layers.LeakyReLU(alpha=0.2)

    self.rl3 = layers.LeakyReLU(alpha=0.2)
  def call(self, inputs):
    shortcut = self.shortcut(inputs)

    inputs = self.conv1(inputs)
    inputs = self.bn1(inputs)
    inputs = self.rl1(inputs)

    inputs = self.conv2(inputs)
    inputs = self.bn2(inputs)
    inputs = self.rl2(inputs)

    inputs = inputs + shortcut
    return self.rl3(inputs)

class EncodeBlock(layers.Layer):
  def __init__(self, filters):
    super().__init__()
    # if downsample:
    #   self.conv1 = layers.Conv2D(filters, kernel_size=3, strides=2, padding='same', kernel_initializer = 'he_normal')
    #   self.shortcut = keras.Sequential([
    #     layers.Conv2D(filters, kernel_size=1, strides=2, kernel_initializer='he_normal'),
    #     layers.BatchNormalization()
    #   ])
    # else:
    #   self.conv1 = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer = 'he_normal')
    #   self.shortcut = keras.Sequential()

    self.conv1 = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer = 'he_normal')
    self.bn1 = layers.BatchNormalization()
    self.rl1 = layers.ReLU()

    self.conv2 = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer = 'he_normal')
    self.bn2 = layers.BatchNormalization()
    self.rl2 = layers.ReLU()

    self.pool = layers.MaxPool2D(pool_size=(2, 2))
  def call(self, inputs):
    # shortcut = self.shortcut(inputs)

    inputs = self.conv1(inputs)
    inputs = self.bn1(inputs)
    inputs = self.rl1(inputs)

    inputs = self.conv2(inputs)
    inputs = self.bn2(inputs)
    inputs = self.rl2(inputs)

    inputs = self.pool(inputs)
    # inputs = inputs + shortcut
    return inputs

class BottleNeckBlock(layers.Layer):
  def __init__(self, dense_shape=400):
    super().__init__()
    self.pool = layers.AveragePooling2D(pool_size=4)
    self.flatten = layers.Flatten()
    self.rl1 = layers.LeakyReLU(alpha=0.2)
    self.fc1 = layers.Dense(dense_shape)
    self.rl2 = layers.LeakyReLU(alpha=0.2)
    self.fc2 = layers.Dense(16384)
    self.reshape = layers.Reshape((8, 8, 8, 32))
  def call(self, inputs):
    inputs = self.pool(inputs)
    inputs = self.flatten(inputs)
    inputs = self.fc1(inputs)
    inputs = self.rl1(inputs)
    inputs = self.fc2(inputs)
    inputs = self.rl2(inputs)
    inputs = self.reshape(inputs)
    return inputs

class DecodeBlock(layers.Layer):
  def __init__(self, filters):
    super().__init__()
    self.deconv = layers.Conv3DTranspose(filters=filters, kernel_size=3, strides=2, padding='same', kernel_initializer = 'he_normal')
    self.bn1 = layers.BatchNormalization()
    self.rl1 = layers.ReLU()

    self.conv = layers.Conv3D(filters=filters, kernel_size=3, strides=1, padding='same', kernel_initializer = 'he_normal')
    self.bn2 = layers.BatchNormalization()
    self.rl2 = layers.ReLU()
  def call(self, inputs):
    inputs = self.deconv(inputs)
    inputs = self.bn1(inputs)
    inputs = self.rl1(inputs)

    inputs = self.conv(inputs)
    inputs = self.bn2(inputs)
    inputs = self.rl2(inputs)
    return inputs

In [17]:
class Generator(keras.Model):
  def __init__(self):
    super().__init__()

    self.layer0 = keras.Sequential([
      layers.Conv2D(32, 7, 2, padding='same'),
      layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
      layers.BatchNormalization(),
      layers.ReLU()
    ], name='layer0')
    
    self.layer1 = keras.Sequential([
      ResBlock(32, downsample=True),
      ResBlock(32, downsample=False)
    ], name='layer1')

    self.layer2 = keras.Sequential([
      ResBlock(64, downsample=True),
      ResBlock(64, downsample=False)
    ], name='layer2')

    self.layer3 = keras.Sequential([
      ResBlock(128, downsample=True),
      ResBlock(128, downsample=False)
    ], name='layer3')

    # self.layer0 = EncodeBlock(16)
    # self.layer1 = EncodeBlock(32)
    # self.layer2 = EncodeBlock(64)
    # self.layer3 = EncodeBlock(128)
    # self.layer31 = EncodeBlock(128)

    # bottleneck
    self.flatten = layers.Flatten()
    self.reshape = layers.Reshape((4, 4, 4, 128))
    # self.bottleneck = BottleNeckBlock()

    self.layer4 = DecodeBlock(64)
    self.layer5 = DecodeBlock(32)
    self.layer6 = DecodeBlock(16)
    self.layer7 = DecodeBlock(8)
    self.layer8 = DecodeBlock(1)

    # self.upsample = layers.UpSampling3D(size=(2, 2, 2))

    # self.softmax = layers.Softmax()

  def call(self, inputs):

    inputs = self.layer0(inputs)
    inputs = self.layer1(inputs)
    inputs = self.layer2(inputs)
    inputs = self.layer3(inputs)
    # inputs = self.layer31(inputs)

    inputs = self.flatten(inputs)
    inputs = self.reshape(inputs)
    # inputs = self.bottleneck(inputs)

    inputs = self.layer4(inputs)
    inputs = self.layer5(inputs)
    inputs = self.layer6(inputs)
    inputs = self.layer7(inputs)
    inputs = self.layer8(inputs)

    # inputs = self.upsample(inputs)

    inputs = tf.keras.activations.tanh(inputs)
    return inputs

In [18]:
# # Build Model
# # initialize model
model = Generator()
# encoder_model.build(input_shape=INPUT_SHAPE)
model.build(input_shape=(40, 256, 256, 3))

# summary model
model.call(layers.Input((256, 256, 3)))
model.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer0 (Sequential)         (None, 64, 64, 32)        4864      
                                                                 
 layer1 (Sequential)         (None, 32, 32, 32)        38688     
                                                                 
 layer2 (Sequential)         (None, 16, 16, 64)        132672    
                                                                 
 layer3 (Sequential)         (None, 8, 8, 128)         527488    
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 reshape (Reshape)           (None, 4, 4, 4, 128)      0         
                                                                 
 decode_block (DecodeBlock)  (None, 8, 8, 8, 64)       33

## Discriminator

In [19]:
class EncodeBlockD(layers.Layer):
  def __init__(self, filters):
    super().__init__()
    self.conv1 = layers.Conv3D(filters=filters, kernel_size=3, strides=1, padding='same', kernel_initializer = 'he_normal')
    # self.bn1 = layers.BatchNormalization()
    self.rl1 = layers.LeakyReLU(alpha=0.2)
    self.conv2 = layers.Conv3D(filters=filters, kernel_size=3, strides=1, padding='same', kernel_initializer = 'he_normal')
    # self.bn2 = layers.BatchNormalization()
    self.rl2 = layers.LeakyReLU(alpha=0.2)
    self.pool = layers.MaxPool3D(pool_size=2, padding='same')
  def call(self, inputs):
    inputs = self.conv1(inputs)
    # inputs = self.bn1(inputs)
    inputs = self.rl1(inputs)
    inputs = self.conv2(inputs)
    # inputs = self.bn2(inputs)
    inputs = self.rl2(inputs)
    inputs = self.pool(inputs)
    return inputs

In [20]:
class Discriminator(keras.Model):
  def __init__(self):
    super().__init__()

    # self.downsample = layers.MaxPool3D(pool_size=(2, 2, 2))

    filters_layers = [4, 16, 32, 64, 128]

    self.layer0 = EncodeBlockD(filters_layers[0])
    self.do0 = layers.Dropout(0.3)
    self.layer1 = EncodeBlockD(filters_layers[1])
    self.do1 = layers.Dropout(0.3)
    self.layer2 = EncodeBlockD(filters_layers[2])
    self.do2 = layers.Dropout(0.3)
    self.layer3 = EncodeBlockD(filters_layers[3])
    self.do3 = layers.Dropout(0.3)
    self.layer4 = EncodeBlockD(filters_layers[4])
    self.do4 = layers.Dropout(0.3)

    self.flatten = layers.Flatten()
    self.fc1 = layers.Dense(10)
    self.fc2 = layers.Dense(1)
    # self.softmax = layers.Softmax()

  def call(self, inputs):
    # inputs = self.downsample(inputs)

    inputs = self.layer0(inputs)
    inputs = self.do0(inputs)
    inputs = self.layer1(inputs)
    inputs = self.do1(inputs)
    inputs = self.layer2(inputs)
    inputs = self.do2(inputs)
    inputs = self.layer3(inputs)
    inputs = self.do3(inputs)
    inputs = self.layer4(inputs)
    inputs = self.do4(inputs)
    inputs = self.flatten(inputs)
    inputs = self.fc1(inputs)
    inputs = self.fc2(inputs)
    # inputs = self.softmax(inputs)
    # inputs = tf.math.sigmoid(inputs)
    return inputs

In [21]:
discriminator = Discriminator()
discriminator.build(input_shape=(None, 256, 256, 256, 1))

# summary model
discriminator.call(layers.Input((256, 256, 256, 1)))
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encode_block_d (EncodeBlock  (None, 128, 128, 128, 4)  548      
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 128, 128, 128, 4)  0         
                                                                 
 encode_block_d_1 (EncodeBlo  (None, 64, 64, 64, 16)   8672      
 ckD)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64, 64, 64, 16)    0         
                                                                 
 encode_block_d_2 (EncodeBlo  (None, 32, 32, 32, 32)   41536     
 ckD)                                                            
                                                     

## Shape Estimator

### Metrics

In [22]:
# reference: https://stackoverflow.com/questions/61673551/working-of-the-earth-mover-loss-method-in-keras-and-input-arguments-data-types
def earth_mover_loss(y_true, y_pred):
  # cdf_true = K.cumsum(tf.cast(y_true, np.float32), axis=-1)
  # cdf_pred = K.cumsum(y_pred, axis=-1)
  # emd = K.sqrt(K.mean(K.square(cdf_true - cdf_pred), axis=-1))
  diff = tf.cast(y_true, np.float32) - y_pred
  emd = K.cumsum(diff, axis=-1)
  return K.abs(K.mean(emd))

In [23]:
class ShapeEstimator(keras.Model):
  def __init__(self, generator, discriminator, gp_weight):
    super().__init__()
    self.generator = generator
    self.discriminator = discriminator
    # self.base_loss = tf.keras.losses.MeanAbsoluteError()
    self.base_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    self.gp_weight = gp_weight

  def compile(self, gen_optimizer, disc_optimizer, metric):
    super().compile()
    self.gen_optimizer = gen_optimizer
    self.disc_optimizer = disc_optimizer
    self.metric = metric

  # loss functions
  # gradient penalty
  def gradient_penalty(self, batch_size, fake_shapes, real_shapes):
    """Calculates the gradient penalty.

    This loss is calculated on an interpolated image
    and added to the discriminator loss.
    """
    # Get the interpolated image
    alpha = tf.random.uniform([batch_size, 1, 1, 1, 1], 0.0, 1.0)
    diff = fake_shapes - real_shapes
    # interpolated = real_shapes + alpha * diff
    interpolated = alpha * diff

    with tf.GradientTape() as gp_tape:
      gp_tape.watch(interpolated)
      # 1. Get the discriminator output for this interpolated image.
      pred = self.discriminator(interpolated, training=True)

    # 2. Calculate the gradients w.r.t to this interpolated image.
    grads = gp_tape.gradient(pred, [interpolated])[0]
    # 3. Calculate the norm of the gradients.
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

  # 1. WGAN
  def generator_loss(self, fake_output):
    return -tf.reduce_mean(fake_output)
  
  def discriminator_loss(self, fake_output, real_output):
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

  # 2. Basic loss
  # def generator_loss(self, fake_output):
  #   return self.base_loss(tf.ones_like(fake_output), fake_output)

  # def discriminator_loss(self, fake_output, real_output):
  #   real_loss = self.base_loss(tf.ones_like(real_output), real_output)
  #   fake_loss = self.base_loss(tf.zeros_like(fake_output), fake_output)
  #   total_loss = real_loss + fake_loss
  #   return total_loss

  # # 3. LS GAN loss
  # # reference: https://github.com/pianomania/LSGAN/blob/master/model.py
  # def generator_loss(self, fake_output):
  #   return tf.reduce_mean(tf.square(fake_output-1))/2

  # def discriminator_loss(self, fake_output, real_output):
  #   return tf.reduce_mean(tf.square(real_output-1) + tf.square(fake_output))/2

  # 4. Original GAN
  # def generator_loss(self, fake_output):
  #   return tf.reduce_mean(tf.math.log(fake_output))

  # def discriminator_loss(self, fake_output, real_output):
  #   return tf.reduce_mean(tf.math.log(real_output)) + tf.reduce_mean(tf.math.log(fake_output))

  @property
  def metrics(self):
    return self.metric

  # @tf.function
  # def train_step_not_skipped(self, data):
  #   # unpack data
  #   image, shape_true = data
  #   batch_size = tf.shape(shape_true)[0]

  #   with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
  #     # predict shape with generator
  #     shape_pred = self.generator(image, training=True)
  #     # predict discrimination results with discriminator
  #     real_output = self.discriminator(shape_true, training=True)
  #     fake_output = self.discriminator(shape_pred, training=True)

  #     # compute loss
  #     gen_loss = self.generator_loss(fake_output)
  #     disc_loss = self.discriminator_loss(fake_output, real_output)

  #     # # compute gradient penalty
  #     if (self.train_counter % 1 == 0):
  #       gp = self.gradient_penalty(batch_size, shape_pred, tf.cast(shape_true, dtype=np.float32))
  #       disc_loss = disc_loss + gp * self.gp_weight


  #   # compute gradient
  #   gen_variables = self.generator.trainable_variables
  #   disc_variables = self.discriminator.trainable_variables

  #   gen_gradients = gen_tape.gradient(gen_loss, gen_variables)
  #   disc_gradients = disc_tape.gradient(disc_loss, disc_variables)

  #   # # update metrics
  #   metric_result = []
  #   # MAE
  #   metric_result.append(self.metric[0](shape_true, shape_pred))
  #   # EM
  #   # metric_result.append(self.metric[1](shape_true, shape_pred))
  #   metric_result.append(0)

  #   # update weights
  #   # if (self.train_counter % 2 == 0) or (float(metric_result) > 0.25):
  #   if (self.train_counter % 1 == 0):
  #     self.disc_optimizer.apply_gradients(zip(disc_gradients, disc_variables))
  #   self.gen_optimizer.apply_gradients(zip(gen_gradients, gen_variables))

  #   # update train counter
  #   self.train_counter += 1

  #   return gen_loss, disc_loss, gp, shape_pred, metric_result


  @tf.function
  def train_step_not_skipped(self, data):
    # unpack data
    image, shape_true = data
    batch_size = tf.shape(shape_true)[0]

    with tf.GradientTape() as disc_tape:
      # predict shape with generator
      shape_pred = self.generator(image, training=True)
      # predict discrimination results with discriminator
      real_output = self.discriminator(shape_true, training=True)
      fake_output = self.discriminator(shape_pred, training=True)

      # compute loss
      # gen_loss = self.generator_loss(fake_output)
      disc_loss = self.discriminator_loss(fake_output, real_output)

      # compute gradient penalty
      if (float(disc_loss) > 1000) or (float(disc_loss) < -1000):
        modified_gp_weight = 200.0
      else:
        modified_gp_weight = self.gp_weight
      gp = self.gradient_penalty(batch_size, shape_pred, tf.cast(shape_true, dtype=np.float32))
      disc_loss = disc_loss + gp * modified_gp_weight


    # compute gradient
    disc_variables = self.discriminator.trainable_variables
    disc_gradients = disc_tape.gradient(disc_loss, disc_variables)

    # update weights
    self.disc_optimizer.apply_gradients(zip(disc_gradients, disc_variables))

    with tf.GradientTape() as gen_tape:
      # predict shape with generator
      shape_pred = self.generator(image, training=True)

      # predict discrimination results with discriminator
      fake_output = self.discriminator(shape_pred, training=True)

      # compute loss
      gen_loss = self.generator_loss(fake_output)

    # compute gradient
    gen_variables = self.generator.trainable_variables
    gen_gradients = gen_tape.gradient(gen_loss, gen_variables)

    # update gradient
    self.gen_optimizer.apply_gradients(zip(gen_gradients, gen_variables))

    # # update metrics
    metric_result = []
    # MAE
    metric_result.append(self.metric[0](shape_true, shape_pred))
    # EM
    # metric_result.append(self.metric[1](shape_true, shape_pred))
    metric_result.append(0)

    return gen_loss, disc_loss, gp * modified_gp_weight, shape_pred, metric_result

  @tf.function
  def test_step(self, data):
    # unpack data
    image, shape_true = data

    # predict
    shape_pred = self.generator(image, training=True)

    # update metrics
    metric_result = []
    metric_result.append(self.metric[0](shape_true, shape_pred))
    metric_result.append(self.metric[1](shape_true, shape_pred))

    return metric_result, shape_pred

In [24]:
from keras.utils.generic_utils import get_custom_objects
def rmse(y_true, y_pred):
  """
  Root Mean Squared Error
  Args:
      y_true ([np.array]): test samples
      y_pred ([np.array]): predicted samples
  Returns:
      [float]: root mean squared error
  """
  return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_true, y_pred))))
  # return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
get_custom_objects().update({"rmse": rmse})

In [25]:
if OPTIMIZER == 'RMSprop':
  gen_optimizer = tf.keras.optimizers.RMSprop()
  disc_optimizer = tf.keras.optimizers.RMSprop()
else:
  gen_optimizer = tf.keras.optimizers.Adam(GEN_LR)
  disc_optimizer = tf.keras.optimizers.Adam(DISC_LR)
generator = Generator()
discriminator = Discriminator()
sketch_estimator = load_model('/content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/sketch_estimator_normal/model')
sketch_estimator_depth = load_model('/content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/sketch_estimator_depth/model')

model = ShapeEstimator(generator=generator, discriminator=discriminator, gp_weight=GP_WEIGHT)
model.compile(gen_optimizer=gen_optimizer, disc_optimizer=disc_optimizer, metric=[tf.keras.metrics.RootMeanSquaredError(), earth_mover_loss])

# Fit Model

In [26]:
# checkpoints
# additional checkpoint load code and temp directory definition
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

gen_checkpoint_path = project_path+"/intermediate/shape_estimator/gen_weights{epoch:02d}.ckpt"
disc_checkpoint_path = project_path+"/intermediate/shape_estimator/disc_weights{epoch:02d}.ckpt"
checkpoint_dir = os.path.dirname(gen_checkpoint_path)

gen_checkpoint = ModelCheckpoint(filepath=gen_checkpoint_path, verbose=1)
disc_checkpoint = ModelCheckpoint(filepath=disc_checkpoint_path, verbose=1)

In [27]:
history = {'gen_loss':[],
      'disc_loss':[],
      'gen_loss_val':[],
      'disc_loss_val':[],
      'gradient_penalty':[],
      'metric_MAE':[],
      'metric_MAE_val':[],
      'metric_EM':[]}

In [28]:

# train
if RUN_MODE == 'train':
  try:
    with open(OUTPUT_PATH+'curr_epoch.txt', 'r') as f:
      curr_epoch = int(f.read())
      print(curr_epoch)
    f.close()
  except FileNotFoundError:
    curr_epoch = 0
    print('File not found, current epoch set as 0.')

  if curr_epoch != 0:
    # loaded_generator = tf.saved_model.load(OUTPUT_PATH+'gen_weights')
    # loaded_discriminator = tf.saved_model.load(OUTPUT_PATH+'disc_weights')
    generator.load_weights(OUTPUT_PATH+'gen_weights%02d.ckpt' % curr_epoch)
    discriminator.load_weights(OUTPUT_PATH+'disc_weights%02d.ckpt' % curr_epoch)
    loaded_generator = generator
    loaded_discriminator = discriminator
    
    loaded_model = ShapeEstimator(generator=loaded_generator, discriminator=loaded_discriminator, gp_weight=GP_WEIGHT)
    loaded_model.compile(gen_optimizer=gen_optimizer, disc_optimizer=disc_optimizer, metric=[tf.keras.metrics.RootMeanSquaredError(), earth_mover_loss])

    model = loaded_model
    print('Model loaded at epoch' + str(curr_epoch))

  import time
  for epoch in range(curr_epoch+1, N_EPOCH):
    start = time.time()
    print('Epoch %02d' % epoch)


    # train batches
    prev_disc_loss = 1
    for step, batch in enumerate(train_loader):
      # # predict sketches
      # if PREDICT_SKETCH == True:
      #   batch = list(batch)
      #   batch[0] = sketch_estimator.predict(batch[0])
      #   batch = tuple(batch)

      # concatenate sketches
      concatenated = np.concatenate((batch[0], batch[1]), axis=3)
      batch = (concatenated, batch[2])

      # train step
      gen_loss, disc_loss, gp, shape_pred, metric = model.train_step_not_skipped(batch)

      # record previous discriminator loss
      prev_disc_loss = float(disc_loss)

      # log
      if step % 50 == 0:
        print('Generator loss at step %d/%d: %.2f' % (step, train_loader.__len__(), gen_loss))
        print('Discriminator loss at step %d/%d: %.2f' % (step, train_loader.__len__(), disc_loss))
        print('Gradient penalty at step %d/%d: %.2f' % (step, train_loader.__len__(), gp))
        print('Metric MAE at step %d/%d: %.2f' % (step, train_loader.__len__(), metric[0]))
        print('Metric EM at step %d/%d: %.2f' % (step, train_loader.__len__(), metric[1]))

        # save history
        history['gen_loss'].append(float(gen_loss))
        history['disc_loss'].append(float(disc_loss))
        history['gradient_penalty'].append(float(gp))
        history['metric_MAE'].append(float(metric[0]))
        history['metric_EM'].append(float(metric[1]))
    
    # validation
    metric_hist = []
    for step, batch in enumerate(validation_loader):
      # # predict sketches
      # if PREDICT_SKETCH == True:
      #   batch = list(batch)
      #   batch[0] = sketch_estimator.predict(batch[0])
      #   batch = tuple(batch)

      # concatenate sketches
      concatenated = np.concatenate((batch[0], batch[1]), axis=3)
      batch = (concatenated, batch[2])

      # validate
      metric, shape_pred = model.test_step(batch)
      metric_hist.append(metric)

      # save shapes
      if step % 50 == 0:
        try:
          os.makedirs(OUTPUT_PATH+'predicted_shapes_validation%02d' % epoch)
        except FileExistsError:
          pass
        with open(OUTPUT_PATH+'predicted_shapes_validation%02d/predicted_shape%02d.npz' % (epoch, step), 'wb+') as f:
          np.save(f, shape_pred)
        f.close()
        with open(OUTPUT_PATH+'predicted_shapes_validation%02d/real_shape%02d.npz' % (epoch, step), 'wb+') as f:
          np.save(f, batch[1])
        f.close()

    # show validation results
    validation_metric_MAE = sum(metric_hist[0]) / len(metric_hist[0])
    print('Validation metric MAE: %.2f' % validation_metric_MAE)
    # validation_metric_EM = sum(metric_hist[1]) / len(metric_hist[1])
    # print('Validation metric EM: %.2f' % validation_metric_EM)

    # save validation results
    history['gen_loss_val'].append(float(gen_loss))
    history['disc_loss_val'].append(float(disc_loss))
    history['metric_MAE_val'].append(float(validation_metric_MAE))

    # save models
    # model.generator.save(OUTPUT_PATH+'gen_weights%02d.ckpt' % epoch)
    # model.discriminator.save(OUTPUT_PATH+'disc_weights%02d.ckpt' % epoch)
    model.generator.save_weights(OUTPUT_PATH+'gen_weights%02d.ckpt' % epoch)
    model.discriminator.save_weights(OUTPUT_PATH+'disc_weights%02d.ckpt' % epoch)

    # save sample predicted shapes and real shapes
    try:
      os.makedirs(OUTPUT_PATH+'predicted_shapes')
    except FileExistsError:
      pass
    with open(OUTPUT_PATH+'predicted_shapes/predicted_shape%02d.npz' % epoch, 'wb+') as f:
      np.save(f, shape_pred)
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes/real_shape%02d.npz' % epoch, 'wb+') as f:
      np.save(f, batch[1])
    f.close()
    print('Shape saved at epoch %02d' % epoch)

    # save checkpoint
    with open(OUTPUT_PATH+'curr_epoch.txt', 'w') as f:
      f.write(str(epoch))
    f.close()

    print('Passed time: '+str(time.time()-start))

In [29]:
if RUN_MODE == 'train':
  # save model history
  with open(OUTPUT_PATH+'history.json', 'w+') as f:
    json.dump(dict(history), f)
  f.close()

  # save models
  generator.save_weights(OUTPUT_PATH+'gen_weights')
  discriminator.save_weights(OUTPUT_PATH+'disc_weights')

# Demonstrate Results

In [30]:
# if RUN_MODE == 'test':
#   metric_hist = []

#   # load model
#   loaded_generator = tf.saved_model.load(OUTPUT_PATH+'gen_weights03.ckpt')
#   loaded_discriminator = tf.saved_model.load(OUTPUT_PATH+'disc_weights03.ckpt')
#   # loaded_generator = tf.keras.models.load_model(OUTPUT_PATH+'gen_weights')
#   # loaded_discriminator = tf.keras.models.load_model(OUTPUT_PATH+'disc_weights')
# generator.load_weights(OUTPUT_PATH+'gen_weights%02d.ckpt' % curr_epoch)
# discriminator.load_weights(OUTPUT_PATH+'disc_weights%02d.ckpt' % curr_epoch)
# loaded_generator = generator
# loaded_discriminator = discriminator
  
#   loaded_model = ShapeEstimator(generator=loaded_generator, discriminator=loaded_discriminator, gp_weight=GP_WEIGHT)
#   loaded_model.compile(gen_optimizer=gen_optimizer, disc_optimizer=disc_optimizer, metric=[tf.keras.losses.MeanAbsoluteError(), earth_mover_loss])

#   # test batches
#   for step, batch in enumerate(test_loader):
#     # predict sketches
#     # batch = list(batch)
#     # batch[0] = sketch_estimator.predict(batch[0])
#     # batch = tuple(batch)
#     metric, shape_pred = loaded_model.test_step(batch)
#     metric_hist.append(metric)

#     # save sample predicted shapes and real shapes
#     if step % 1 == 0:
#       try:
#         os.makedirs(OUTPUT_PATH+'predicted_shapes_test')
#       except FileExistsError:
#         pass
#       with open(OUTPUT_PATH+'predicted_shapes_test/predicted_shape%02d.npz' % step, 'wb+') as f:
#         np.save(f, shape_pred)
#       f.close()
#       with open(OUTPUT_PATH+'predicted_shapes_test/real_shape%02d.npz' % step, 'wb+') as f:
#         np.save(f, batch[1])
#       f.close()
#       with open(OUTPUT_PATH+'predicted_shapes_test/image%02d.npz' % step, 'wb+') as f:
#         np.save(f, batch[0])
#       f.close()
#       print('Shape saved at step %02d' % step)
#   avg_metric_MAE = sum(metric_hist[0]) / len(metric_hist[0])
#   avg_metric_EM = sum(metric_hist[1]) / len(metric_hist[1])
#   print('Average metric MAE: ' + str(avg_metric_MAE))
#   print('Average metric EM: ' + str(avg_metric_EM))

In [31]:
metric_hist = []

# # load model
# loaded_generator = tf.saved_model.load(OUTPUT_PATH+'gen_weights03.ckpt')
# loaded_discriminator = tf.saved_model.load(OUTPUT_PATH+'disc_weights03.ckpt')
# # loaded_generator = tf.keras.models.load_model(OUTPUT_PATH+'gen_weights')
# # loaded_discriminator = tf.keras.models.load_model(OUTPUT_PATH+'disc_weights')
curr_epoch = 10
generator.load_weights(OUTPUT_PATH+'gen_weights%02d.ckpt' % curr_epoch)
discriminator.load_weights(OUTPUT_PATH+'disc_weights%02d.ckpt' % curr_epoch)
loaded_generator = generator
loaded_discriminator = discriminator

loaded_model = ShapeEstimator(generator=loaded_generator, discriminator=loaded_discriminator, gp_weight=GP_WEIGHT)
loaded_model.compile(gen_optimizer=gen_optimizer, disc_optimizer=disc_optimizer, metric=[tf.keras.metrics.RootMeanSquaredError(), earth_mover_loss])

# loaded_model = model

# test batches
count = 0
for step, batch in enumerate(test_loader):
  # save original image
  if step % 10 == 0:
    try:
      os.makedirs(OUTPUT_PATH+'predicted_shapes_test')
    except FileExistsError:
      pass
    with open(OUTPUT_PATH+'predicted_shapes_test/real_raw_image%02d.npz' % count, 'wb+') as f:
      np.save(f, batch[3])
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/real_normal_image%02d.npz' % count, 'wb+') as f:
      np.save(f, batch[0])
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/real_depth_image%02d.npz' % count, 'wb+') as f:
      np.save(f, batch[1])
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/predicted_normal_image%02d.npz' % count, 'wb+') as f:
      np.save(f, sketch_estimator.predict(batch[3]))
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/predicted_depth_image%02d.npz' % count, 'wb+') as f:
      np.save(f, sketch_estimator_depth.predict(batch[3]))
    f.close()

  # predict sketches
  # batch = list(batch)
  # batch[0] = sketch_estimator.predict(batch[0])
  # batch = tuple(batch)

  # concatenate sketches
  concatenated = np.concatenate((batch[0], batch[1]), axis=3)
  # concatenated = np.concatenate((sketch_estimator.predict(batch[3]), sketch_estimator_depth.predict(batch[3])), axis=3)
  batch = (concatenated, batch[2])

  metric, shape_pred = loaded_model.test_step(batch)
  metric_hist.append(metric)

  # save sample predicted shapes and real shapes
  if step % 10 == 0:
    try:
      os.makedirs(OUTPUT_PATH+'predicted_shapes_test')
    except FileExistsError:
      pass
    with open(OUTPUT_PATH+'predicted_shapes_test/predicted_shape%02d.npz' % count, 'wb+') as f:
      np.save(f, shape_pred)
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/real_shape%02d.npz' % count, 'wb+') as f:
      np.save(f, batch[1])
    f.close()
    # with open(OUTPUT_PATH+'predicted_shapes_test/predicted_image%02d.npz' % count, 'wb+') as f:
    #   np.save(f, batch[0])
    # f.close()
    print('Shape saved at step %02d' % step)
    count += 1
avg_metric_MAE = sum(metric_hist[0]) / len(metric_hist[0])
avg_metric_EM = sum(metric_hist[1]) / len(metric_hist[1])
print('Average metric MAE: ' + str(avg_metric_MAE))
print('Average metric EM: ' + str(avg_metric_EM))

Shape saved at step 00
Shape saved at step 10
Shape saved at step 20
Shape saved at step 30
Shape saved at step 40
Shape saved at step 50
Shape saved at step 60
Shape saved at step 70
Shape saved at step 80
Shape saved at step 90
Shape saved at step 100
Shape saved at step 110
Shape saved at step 120
Shape saved at step 130
Shape saved at step 140
Shape saved at step 150
Shape saved at step 160
Shape saved at step 170
Average metric MAE: tf.Tensor(0.22355634, shape=(), dtype=float32)
Average metric EM: tf.Tensor(0.19753934, shape=(), dtype=float32)


In [ ]:
# list all data in history
with open(OUTPUT_PATH+'history.json', 'r') as f:
  history = json.load(f)
f.close()
# summarize history for accuracy
plt.figure(figsize=(8,5))
plt.plot(history['gen_loss'])
plt.plot(history['disc_loss'])
plt.title('Loss History of Net')
plt.ylabel('loss')
plt.xlabel('time')
plt.xticks(np.arange(0, len(history['gen_loss']), 100.0))
# plt.yticks(np.arange(0.0, max(max(history['disc_loss']), max(history['gen_loss'])), 1.0))
plt.ylim(-1000, 1000)
plt.legend(['gen', 'disc'], loc='upper left')
plt.savefig(OUTPUT_PATH+'losses.jpg')
plt.show()
# summarize history for metric
plt.figure(figsize=(8,5))
plt.plot(history['metric_MAE'])
plt.title('Metric History of Net')
plt.ylabel('metric')
plt.xlabel('epoch')
plt.xticks(np.arange(0, len(history['gen_loss']), 100.0))
plt.yticks(np.arange(0.0, 0.5, 0.1))
plt.ylim(0.0, 0.5)
plt.legend(['metric'], loc='upper left')
plt.savefig(OUTPUT_PATH+'metrics.jpg')
plt.show()